# Setup 

In [1]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get update


apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,080 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nv

In [2]:
!pip install vizdoom
!pip install ray 
!pip install ray['rllib']
!pip install Ipython --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 12.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for vizdoom: filename=vizdoom-1.1.13-cp37-cp37m-linux_x86_64.whl size=14101154 sha256=41af410be95cc4302f5763e2d5c0278e618b6fc8b5bb5953c36f98c71fe90299
  Stored in directory: /root/.cache/pip/wheels/ac/37/ae/8e648023f66bb4c473701f94ce126032ff39ad9759ca0645a7
Successfully built vizdoom
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 134 kB/s 
     |████████████████████████████████| 8.8 MB 63.1 MB/s 
     |████████████████████████████████| 4.1 MB 55.6 MB/s 
     |████████████████████████████████| 468 kB 60.2 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.48.1
    Un

In [3]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project'
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(system_path)

Mounted at /content/drive


In [4]:
from src.vizdoom_gym.envs.VizDoomEnv import VizdoomEnv
from src.vizdoom_gym.envs.VizDoomEnv_def import VizDoomVeryDenseReward

In [5]:
from ray.tune.registry import register_env
import gym
import os
import ray
import ray.rllib.agents.ppo as ppo
import shutil
import torch

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: ", device, "\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

device:  cuda:0 

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


# Initialize Ray

In [7]:
#need this to load vizdoom module 
system_path = '/content/drive/MyDrive/GitHub/INM363-Project/src' 
sys.path.append(system_path)

#need this to use gpu on ray 
os.environ['PYTHONPATH'] = '/content/drive/MyDrive/GitHub/INM363-Project' 
os.environ['PYTHONPATH']

'/content/drive/MyDrive/GitHub/INM363-Project'

In [8]:

#chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/easy_dense"
#shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)


ray.shutdown()
print("Shutdown ray")

# start Ray -- add `local_mode=True` here for debugging
ray.init(ignore_reinit_error=True,  num_cpus =2, num_gpus = 1) #local_mode=True,

#ray.init(num_cpus= 2, num_gpus=1)

print("Initialized ray")

# register the custom environment
select_env = "VizDoomVeryDenseReward-v0"

register_env(select_env, lambda config: VizDoomVeryDenseReward())
#register_env(select_env, lambda config: VizdoomEnv())

print("registered environment")


Shutdown ray


2022-09-24 22:28:37,577	INFO worker.py:1518 -- Started a local Ray instance.


Initialized ray
registered environment


Training Config

In [9]:
# configure the environment and create agent
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
config["num_workers"] = 1
config["framework"] = "torch"
config["model"] = {"dim": 42, 
                   "grayscale": True,
                   }
config["num_gpus"] = 1
config["preprocessor_pref"] = "rllib"
config['explore'] = True 
config['batch_mode'] = 'complete_episodes'


#config["horizon"] = 50
agent = ppo.PPOTrainer(config, env=select_env)

print("created agent")

2022-09-24 22:28:38,382	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-09-24 22:28:38,386	INFO algorithm.py:354 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/usr/local/lib/python3.7/dist-packages/ray/_private/ray_option_utils.py:273: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  stacklevel=1,


(RolloutWorker pid=8648) config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
(RolloutWorker pid=8648) scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/train/new_sparse_rs.wad
(RolloutWorker pid=8648) episode timeout: 800
(RolloutWorker pid=8648) screen resolution: 320X240


(RolloutWorker pid=8648) 2022-09-24 22:28:46,076	WARNING env.py:143 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-09-24 22:28:58,773	INFO trainable.py:164 -- Trainable.setup took 20.392 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-09-24 22:28:58,777	WARNING util.py:65 -- Install gputil for GPU system monitoring.


created agent


Training loop

In [10]:
import pandas as pd
import time 

cols = ["checkpoint", "eps_reward_min", "eps_reward_mean", "eps_reward_max", "eps_len_mean", "episodes_this_iter"]
results_df = pd.DataFrame(columns = cols) 

In [11]:
chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse"
chkpt_file  = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse/checkpoint_000200"
agent.restore(chkpt_file)

2022-09-24 22:29:00,886	INFO trainable.py:669 -- Restored on 172.28.0.2 from checkpoint: /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse/checkpoint_000200
2022-09-24 22:29:00,889	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 200, '_timesteps_total': None, '_time_total': 17135.15302848816, '_episodes_total': 4984}


In [12]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f}"
start_n = 200
n_iter = 600

print("started training loop")
time_start = time.time() 

# train a policy with RLlib using PPO
for n in range(start_n, n_iter):
  
    result = agent.train()

    #print(i)
    #print(result)
    if (n+1) % 20 == 0 or n == 0: 
      chkpt_file = agent.save(chkpt_root)
      print(f"Saved checkpoint {n+1} at {chkpt_file}")
    #chkpt_file = "not saving checkpoints"

    print(status.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"]
    ))

    #save metrics
    row = {'checkpoint': n+1,
       "eps_reward_min": result["episode_reward_min"],
       "eps_reward_mean": result["episode_reward_mean"],
       "eps_reward_max": result["episode_reward_max"],
       "eps_len_mean": result["episode_len_mean"],
       "episodes_this_iter": result["episodes_this_iter"]
       }
    results_df = results_df.append(row, ignore_index = True)


print(f"Total time elapsed: {(time.time()-time_start)/60}")

print("ending training loop")

ray.shutdown()
print("shutdown ray")

started training loop


(RolloutWorker pid=8648) 2022-09-24 22:29:16,675	WARNING deprecation.py:48 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!


201 reward   0.20/  4.04/ 11.59 len 174.09
202 reward  -0.80/  4.72/ 11.59 len 158.41
203 reward  -0.80/  4.29/ 11.76 len 160.58
204 reward  -0.80/  4.01/ 11.76 len 158.68
205 reward  -0.80/  4.18/ 11.76 len 147.27
206 reward  -0.80/  4.47/ 11.77 len 141.85
207 reward  -0.80/  4.34/ 11.77 len 144.32
208 reward  -0.80/  4.44/ 11.77 len 144.75
209 reward  -0.80/  4.56/ 11.74 len 149.28
210 reward  -0.80/  4.04/ 11.74 len 154.40
211 reward  -0.80/  4.36/ 11.74 len 152.57
212 reward  -0.80/  3.79/ 11.68 len 160.88
213 reward  -0.80/  3.94/ 11.68 len 155.59
214 reward  -0.80/  3.97/ 11.68 len 152.78
215 reward  -0.80/  5.48/ 11.68 len 132.30
216 reward  -0.80/  5.54/ 11.68 len 128.97
217 reward  -0.80/  5.42/ 11.68 len 131.78
218 reward  -0.80/  4.79/ 11.61 len 137.10
219 reward  -0.80/  4.66/ 11.64 len 143.90
Saved checkpoint 220 at /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse/checkpoint_000220
220 reward  -0.80/  5.22/ 11.85 len 138.08
221 reward  -0.80/  6.24

# save results file

In [13]:
from pathlib import Path 

fname = chkpt_root + '/result.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
results_df.to_csv(fpath)
print(f"Saved results file to {fname}")


Saved results file to /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse/result.csv


In [ ]:
results_df.head()

,checkpoint,eps_reward_min,eps_reward_mean,eps_reward_max,eps_len_mean,episodes_this_iter
0,1.0,-0.8,-0.800000,-0.800,200.000000,20.0
1,2.0,-0.8,-0.128537,9.730,196.780488,21.0
2,3.0,-0.8,0.315328,9.954,190.734375,23.0
3,4.0,-0.8,0.464616,9.986,189.116279,22.0
4,5.0,-0.8,0.337570,9.986,190.640000,20.0


In [20]:
import pandas as pandas
import os 
from pathlib import Path 


chkpt_root = "/content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse" 
results_pth = chkpt_root + '/' + 'result_1.csv'
df1 = pd.read_csv(results_pth) 
df1.head()

results_pth = chkpt_root + '/' + 'result_2.csv'
df2 = pd.read_csv(results_pth) 
df2.head()

frames = [df1, df2]
df = pd.concat(frames, ignore_index = True)
df.tail()

,Unnamed: 0,checkpoint,eps_reward_min,eps_reward_mean,eps_reward_max,eps_len_mean,episodes_this_iter
595,395,596.0,-0.8,9.88463,12.704,64.21,64.0
596,396,597.0,-0.8,10.39622,12.708,58.79,76.0
597,397,598.0,-0.8,10.14392,12.700,59.35,70.0
598,398,599.0,-0.8,10.04536,12.666,61.57,65.0
599,399,600.0,-0.8,10.52385,12.670,61.96,67.0


In [21]:
fname = chkpt_root + '/result.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
df.to_csv(fpath)
print(f"Saved results file to {fname}")

Saved results file to /content/drive/MyDrive/GitHub/INM363-Project/model_checkpoints/ppo/sparse/result.csv


In [ ]:




results_pth = chkpt_root + '/' + 'result_2.csv'
df2 = pd.read_csv(results_pth) 
df2.head()

results_pth = chkpt_root + '/' + 'result_3.csv'
df3 = pd.read_csv(results_pth) 
df3.head()

frames = [df1, df2, df3]
df = pd.concat(frames, ignore_index = True)
df.tail()


fname = chkpt_root + '/result.csv'
fpath = Path(fname)
fpath.parent.mkdir(parents=True, exist_ok = True)
df.to_csv(fpath)
print(f"Saved results file to {fname}")